# Load in extracted features

In [1]:
# open the every file in a directory and print the first line of each file
import os
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN


In [12]:
# load in the features
features_train = np.array([])
for file in os.listdir('features_train'):
    with open('features_train/' + file) as f:
        df = pd.read_csv(f)
        features_train = np.column_stack((features_train, df.values)) if features_train.size else df.values

# load the labels
train = pd.read_csv('train.csv')
y_train = train['Genre']
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
# normalize the data
features_train = preprocessing.scale(features_train)

ValueError: n_components=100 must be between 0 and min(n_samples, n_features)=61 with svd_solver='full'

# Load in test Data


In [13]:
features_test = np.array([])
for file in os.listdir('features_test'):
    with open('features_test/' + file) as f:
        df = pd.read_csv(f)
        features_test = np.column_stack((features_test, df.values)) if features_test.size else df.values

# scale the test data
features_test = preprocessing.scale(features_test)


# Logistic Regression

In [14]:

logreg = LogisticRegression(max_iter=300)
logreg.fit(features_train, y_train)
predictions = logreg.predict(features_train)
# Calculate the accuracy
accuracy = accuracy_score(y_train, predictions)
print("Logistic Regression accuracy:", accuracy)


Logistic Regression accuracy: 0.86625


# Predict with Logistic Regression and save to CSV

In [16]:

# run the model on the test data
predictions = logreg.predict(features_test)
predictions = label_encoder.inverse_transform(predictions)
# save the predictions to a csv file
df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictions.csv', index=False)

# Neural Network

In [23]:


model = Sequential()
model.add(Dense(80, activation='relu', input_shape = (61,), kernel_regularizer='l2', bias_regularizer='l2'))

model.add(Dense(256, activation='relu'))

model.add(Dense(60, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(features_train, y_train, epochs=20, batch_size=28)

loss, accuracy = model.evaluate(features_train, y_train)
print('Neural Network accuracy:', accuracy)

Epoch 1/20


29/29 [==============================] - 2s 3ms/step - loss: 3.2000 - accuracy: 0.3237
Epoch 2/20
29/29 [==============================] - 0s 3ms/step - loss: 1.9160 - accuracy: 0.5562
Epoch 3/20
29/29 [==============================] - 0s 3ms/step - loss: 1.6299 - accuracy: 0.6375
Epoch 4/20
29/29 [==============================] - 0s 2ms/step - loss: 1.4462 - accuracy: 0.6812
Epoch 5/20
29/29 [==============================] - 0s 3ms/step - loss: 1.3180 - accuracy: 0.7237
Epoch 6/20
29/29 [==============================] - 0s 3ms/step - loss: 1.2152 - accuracy: 0.7462
Epoch 7/20
29/29 [==============================] - 0s 2ms/step - loss: 1.1220 - accuracy: 0.7775
Epoch 8/20
29/29 [==============================] - 0s 2ms/step - loss: 1.0503 - accuracy: 0.7937
Epoch 9/20
29/29 [==============================] - 0s 2ms/step - loss: 0.9851 - accuracy: 0.7887
Epoch 10/20
29/29 [==============================] - 0s 2ms/step - loss: 0.9303 - accuracy: 0.8150
Epoch 11/20
29/29 [===========

# Predict with Neural Network and save to CSV

In [24]:
predictions = model.predict(features_test)
predictions = np.argmax(predictions, axis=1)
predictions = label_encoder.inverse_transform(predictions)


df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictions.csv', index=False)

7/7 [==============================] - 0s 3ms/step
